<h1>Extracting results</h1>
<p>Here we extract data required to plot results from the data generated by 
<a href="https://github.com/monarch-initiative/negativeExampleSelection/blob/main/scripts/runSTRING.py">runString.py</a> and
    <a href="https://github.com/monarch-initiative/negativeExampleSelection/blob/main/scripts/runSli.py">runSlipy</a>. Note that we also provide scripts for running these two Python scripts in a SLURM environment,
 <a href="https://github.com/monarch-initiative/negativeExampleSelection/blob/main/scripts/runStringWithSlurm.py">runStringWithSlurm.sh</a> and
    <a href="https://github.com/monarch-initiative/negativeExampleSelection/blob/main/scripts/runSliWithSlurm.sh">runSliWithSlurm.sh</a> </p>
<p>The result of analysis with these scripts are output files
  <a href="https://github.com/monarch-initiative/negativeExampleSelection/blob/main/scripts/sli_only_results.csv">sli_only_results.csv</a>  which are used in this Notebook to calculate the means and standard deviations for the computational experiments.
</p>

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
from glob import glob
import json

<h3>collecting data</h3>
<p>The following function renames some columns, recodes some True/False columns using string values, and outputs a subset of columns into a CSV file that can be used for plotting</p>

In [2]:
def extract_core_results(df):
    df["evaluation_negative_sampling_method"] = ["DANS" if use_scale_free_distribution else "UNS"
        for use_scale_free_distribution in df.use_scale_free_distribution]
    df["model_negative_sampling_method"] = [ "DANS" if use_scale_free_distribution else "UNS"
          for use_scale_free_distribution in df["('model_parameters', 'use_scale_free_distribution')"]                                  ]
    columns = ["evaluation_mode", "features_names","evaluation_negative_sampling_method",
               "model_negative_sampling_method", "accuracy",  "balanced_accuracy",
               "false_discovery_rate","matthews_correlation_coefficient",
           "precision", "recall", "specificity", "f1_score", "auroc", "auprc"]
    return df[columns].copy()

<H2>Aggregating Results</H2>
<p>Here, we calculate the mean and standard error of the
    balanced accuracy, false discovery rate, matthews correlation coefficient,
    F1 score, AUROC, and AUPRC.</p>

In [3]:
def get_mean_and_sd(df):
    """
    df should be one of string_results or sli_results
    """
    # We are just interested in the following seven graph/random walk methods
    graph_methods = {'First-order LINE', 'DeepWalk SkipGram', 'Walklets CBOW', 'HOPE',
                    'Second-order LINE','DeepWalk CBOW',    'Walklets SkipGram'}
    df = df[df['features_names'].isin(graph_methods)]
    # Rename columns for conciseness
    df =  df.rename(columns={"evaluation_negative_sampling_method": "evaluation",
                    "evaluation_mode": "mode","features_names":"methods",
                    "model_negative_sampling_method": "model_neg_sampling"}, errors="raise")
    # Define the columns that we want to calculate mean and sem for
    evaluation_d = {'balanced_accuracy':['mean','std'],
                'false_discovery_rate':['mean','std'],
                'matthews_correlation_coefficient':['mean','std'],
                'f1_score':['mean','std'],
                'auroc':['mean','std'],
                'auprc':['mean','std']}
    # Calculate mean and standard dev
    df2 =  df.groupby(['methods','mode','evaluation','model_neg_sampling']).agg(evaluation_d).reset_index()
    # Make a new column for convenience in planning
    df2["approach"] = df2["evaluation"] + " (" + df2["mode"] + ")"
    # rename column nanes for better legibility
    columns = ["methods",'mode','evaluation', "model_neg_sampling",
            "balanced_acc.mean","balanced_acc.std",
            "FDR.mean", "FDR.std",
            "MCC.mean", "MCC.std",
            "F1.mean", "F1.std",
            "AUROC.mean", "AUROC.std",
            "AUPRC.mean","AUPRC.std",
            "approach"]
    df2.columns = columns
    return df2

<h3>SLI synthetic lethality results</h3>

In [5]:
#df = pd.read_csv("sli_only_results_jul23.tsv", sep='\t')

In [26]:
! ls

KG-Monarch.ipynb           graphs
KGIDG-network.ipynb        idg_stats.csv
LICENSE                    kg_idg_negative_select.tsv
README.rst                 modelParameterTable.ipynb
SCRATH.py                  monarch-kg.tar.gz
SLDBHomoSapiens            monarch-kg.tar.gz.1
SliNetwork.ipynb           results
Untitled.ipynb             scripts
barplots                   sli_results.csv
degreeBias.ipynb           sli_stats.csv
extractingResults.ipynb    string_results.csv
grape_env


In [ ]:
sli_results = extract_core_results(df)

In [ ]:
sli_results.head()

,evaluation_mode,features_names,evaluation_negative_sampling_method,model_negative_sampling_method,accuracy,balanced_accuracy,false_discovery_rate,matthews_correlation_coefficient,precision,recall,specificity,f1_score,auroc,auprc
0,train,First-order LINE,DANS,DANS,0.676118,0.676118,0.312645,0.352871,0.687355,0.646129,0.706107,0.666105,0.736399,0.706748
1,test,First-order LINE,DANS,DANS,0.591653,0.591653,0.384298,0.187399,0.615702,0.487725,0.695581,0.544292,0.645766,0.637471
2,train,First-order LINE,DANS,UNS,0.771810,0.771810,0.206937,0.545056,0.793063,0.735551,0.808070,0.763225,0.823869,0.837614
3,test,First-order LINE,DANS,UNS,0.698854,0.698854,0.238710,0.409574,0.761290,0.579378,0.818331,0.657993,0.731430,0.748987
4,train,First-order LINE,DANS,DANS,0.681025,0.681025,0.294808,0.364588,0.705192,0.622137,0.739913,0.661066,0.744830,0.727736


In [ ]:
sli_stats = get_mean_and_sd(sli_results)

In [ ]:
sli_stats.head()

methods   mode evaluation model_neg_sampling balanced_accuracy  \
                                                                   mean   
0  DeepWalk CBOW   test       DANS               DANS          0.613175   
1  DeepWalk CBOW   test       DANS                UNS          0.803110   
2  DeepWalk CBOW   test        UNS               DANS          0.619108   
3  DeepWalk CBOW   test        UNS                UNS          0.866162   
4  DeepWalk CBOW  train       DANS               DANS          0.806543   

            false_discovery_rate           matthews_correlation_coefficient  \
        std                 mean       std                             mean   
0  0.014518             0.286633  0.034964                         0.256424   
1  0.017240             0.181817  0.026000                         0.607548   
2  0.024639             0.243850  0.039998                         0.281914   
3  0.015686             0.061168  0.021558                         0.742785   
4  0.029446             0.167236  0.032533                         0.615213   

             f1_score               auroc               auprc            \
        std      mean       std      mean       std      mean       std   
0  0.033425  0.495685  0.029277  0.679624  0.034233  0.674321  0.028182   
1  0.034897  0.798539  0.017842  0.877389  0.016808  0.877670  0.016867   
2  0.051737  0.477817  0.049237  0.702873  0.040953  0.698838  0.033761   
3  0.031252  0.854050  0.017742  0.945937  0.016853  0.949158  0.015656   
4  0.059065  0.798591  0.030886  0.884297  0.030673  0.881733  0.024366   

       approach  
                 
0   DANS (test)  
1   DANS (test)  
2    UNS (test)  
3    UNS (test)  
4  DANS (train)

In [ ]:
sli_stats = rename_columns(sli_stats)
sli_stats.head()

,methods,mode,evaluation,model_neg_sampling,balanced_acc.mean,balanced_acc.std,FDR.mean,FDR.std,MCC.mean,MCC.std,F1.mean,F1.std,AUROC.mean,AUROC.std,AUPRC.mean,AUPRC.std,approach
0,DeepWalk CBOW,test,DANS,DANS,0.613175,0.014518,0.286633,0.034964,0.256424,0.033425,0.495685,0.029277,0.679624,0.034233,0.674321,0.028182,DANS (test)
1,DeepWalk CBOW,test,DANS,UNS,0.803110,0.017240,0.181817,0.026000,0.607548,0.034897,0.798539,0.017842,0.877389,0.016808,0.877670,0.016867,DANS (test)
2,DeepWalk CBOW,test,UNS,DANS,0.619108,0.024639,0.243850,0.039998,0.281914,0.051737,0.477817,0.049237,0.702873,0.040953,0.698838,0.033761,UNS (test)
3,DeepWalk CBOW,test,UNS,UNS,0.866162,0.015686,0.061168,0.021558,0.742785,0.031252,0.854050,0.017742,0.945937,0.016853,0.949158,0.015656,UNS (test)
4,DeepWalk CBOW,train,DANS,DANS,0.806543,0.029446,0.167236,0.032533,0.615213,0.059065,0.798591,0.030886,0.884297,0.030673,0.881733,0.024366,DANS (train)


In [ ]:
sli_stats.to_csv("sli_stats.csv")

## KG-IDG results

In [6]:
df_idg = pd.read_csv("scripts/results/kg_idg_negative_select_oct23.tsv", sep="\t")

In [7]:
idg_results = extract_core_results(df_idg)
idg_results.head()

,evaluation_mode,features_names,evaluation_negative_sampling_method,model_negative_sampling_method,accuracy,balanced_accuracy,false_discovery_rate,matthews_correlation_coefficient,precision,recall,specificity,f1_score,auroc,auprc
0,train,First-order LINE,DANS,DANS,0.489124,0.489124,1.0,-0.104861,0.0,0.0,0.978248,0.0,0.480314,0.467894
1,test,First-order LINE,DANS,DANS,0.489430,0.489430,1.0,-0.103357,0.0,0.0,0.978860,0.0,0.478463,0.466092
2,train,First-order LINE,DANS,DANS,0.489124,0.489124,1.0,-0.104861,0.0,0.0,0.978248,0.0,0.480314,0.467894
3,test,First-order LINE,DANS,DANS,0.489430,0.489430,1.0,-0.103357,0.0,0.0,0.978860,0.0,0.478463,0.466092
4,train,First-order LINE,DANS,DANS,0.487898,0.487898,1.0,-0.110680,0.0,0.0,0.975797,0.0,0.484054,0.465875


In [8]:
idg_stats = get_mean_and_sd(idg_results)


In [ ]:
idg_stats = rename_columns(idg_stats)
idg_stats.head()

,methods,mode,evaluation,model_neg_sampling,balanced_acc.mean,balanced_acc.std,FDR.mean,FDR.std,MCC.mean,MCC.std,F1.mean,F1.std,AUROC.mean,AUROC.std,AUPRC.mean,AUPRC.std,approach
0,DeepWalk CBOW,test,DANS,DANS,0.488419,0.003329,1.0,0.0,-0.107239,0.015279,0.0,0.0,0.470524,0.005613,0.461260,0.005005,DANS (test)
1,DeepWalk CBOW,test,UNS,DANS,0.499954,0.000140,1.0,0.0,-0.021442,0.000000,0.0,0.0,0.572750,0.005217,0.690399,0.008049,UNS (test)
2,DeepWalk CBOW,train,DANS,DANS,0.489445,0.000900,1.0,0.0,-0.103193,0.004433,0.0,0.0,0.479023,0.003370,0.465216,0.002695,DANS (train)
3,DeepWalk CBOW,train,UNS,DANS,0.499985,0.000047,1.0,0.0,-0.012378,0.000000,0.0,0.0,0.593047,0.002588,0.706099,0.002453,UNS (train)
4,DeepWalk SkipGram,test,DANS,DANS,0.488419,0.003329,1.0,0.0,-0.107239,0.015279,0.0,0.0,0.470524,0.005613,0.461260,0.005005,DANS (test)


In [9]:
idg_stats.to_csv("idg_stats.csv")

# Disease to phenotype

In [10]:
d2p_df = pd.read_csv("scripts/results/d2p_negative_select_oct30.tsv", sep='\t')

In [11]:
d2p_results = extract_core_results(d2p_df)
d2p_results.head()

,evaluation_mode,features_names,evaluation_negative_sampling_method,model_negative_sampling_method,accuracy,balanced_accuracy,false_discovery_rate,matthews_correlation_coefficient,precision,recall,specificity,f1_score,auroc,auprc
0,train,First-order LINE,DANS,DANS,0.45712,0.45712,0.630078,-0.115572,0.369922,0.121944,0.792296,0.183423,0.506047,0.469213
1,test,First-order LINE,DANS,DANS,0.45696,0.45696,0.630300,-0.115910,0.369700,0.122118,0.791801,0.183593,0.504375,0.468585
2,train,First-order LINE,DANS,DANS,0.45712,0.45712,0.630078,-0.115572,0.369922,0.121944,0.792296,0.183423,0.506047,0.469213
3,test,First-order LINE,DANS,DANS,0.45696,0.45696,0.630300,-0.115910,0.369700,0.122118,0.791801,0.183593,0.504375,0.468585
4,train,First-order LINE,DANS,DANS,0.45627,0.45627,0.631328,-0.117390,0.368672,0.122763,0.789776,0.184193,0.505699,0.468868


In [15]:
d2p_stats = get_mean_and_sd(d2p_results)
d2p_stats.head(20)

,methods,mode,evaluation,model_neg_sampling,balanced_acc.mean,balanced_acc.std,FDR.mean,FDR.std,MCC.mean,MCC.std,F1.mean,F1.std,AUROC.mean,AUROC.std,AUPRC.mean,AUPRC.std,approach
0,DeepWalk CBOW,test,DANS,DANS,0.459032,0.001715,0.623669,0.004586,-0.110201,0.004338,0.187270,0.002838,0.503008,0.000890,0.468286,0.000560,DANS (test)
1,DeepWalk CBOW,test,UNS,DANS,0.561071,0.001097,0.019775,0.000645,0.250269,0.002484,0.221177,0.003460,0.921597,0.002609,0.914750,0.002738,UNS (test)
2,DeepWalk CBOW,train,DANS,DANS,0.459112,0.001601,0.623517,0.004296,-0.110018,0.004054,0.187246,0.002709,0.504967,0.000607,0.469054,0.000262,DANS (train)
3,DeepWalk CBOW,train,UNS,DANS,0.561047,0.001033,0.019849,0.000386,0.250198,0.002275,0.221114,0.003308,0.923801,0.002300,0.916023,0.002488,UNS (train)
4,DeepWalk SkipGram,test,DANS,DANS,0.459032,0.001715,0.623669,0.004586,-0.110201,0.004338,0.187270,0.002838,0.503008,0.000890,0.468286,0.000560,DANS (test)
5,DeepWalk SkipGram,test,UNS,DANS,0.561071,0.001097,0.019775,0.000645,0.250269,0.002484,0.221177,0.003460,0.921597,0.002609,0.914750,0.002738,UNS (test)
6,DeepWalk SkipGram,train,DANS,DANS,0.459112,0.001601,0.623517,0.004296,-0.110018,0.004054,0.187246,0.002709,0.504967,0.000607,0.469054,0.000262,DANS (train)
7,DeepWalk SkipGram,train,UNS,DANS,0.561047,0.001033,0.019849,0.000386,0.250198,0.002275,0.221114,0.003308,0.923801,0.002300,0.916023,0.002488,UNS (train)
8,First-order LINE,test,DANS,DANS,0.459032,0.001715,0.623669,0.004586,-0.110201,0.004338,0.187270,0.002838,0.503008,0.000890,0.468286,0.000560,DANS (test)
9,First-order LINE,test,UNS,DANS,0.561071,0.001097,0.019775,0.000645,0.250269,0.002484,0.221177,0.003460,0.921597,0.002609,0.914750,0.002738,UNS (test)


In [14]:
d2p_stats.to_csv("d2p.tsv", sep="\t")